In [30]:
import pipeline_functions
from importlib import reload
import pandas
import numpy as np
from utils import *
from shutil import copyfile, copytree, rmtree
import glob
from scipy.misc import imread
import cv2
from sklearn import cluster
import h_gen
reload(pipeline_functions)

<module 'pipeline_functions' from 'C:\\Users\\yolo\\Desktop\\fish\\pipeline_functions.py'>

In [2]:
submission_path = "./Submission/submission_csv/submission__2017-01-21-02-26.csv"
test_path = "./Submission/test_data/"

In [43]:
table = pandas.read_csv(submission_path)
numpy_table = table.values

In [46]:
# We convert it to a dicctionary for easier access:

dic_submission = {}

for i, array in enumerate(table.values):
    dic_submission[array[0]] = i

In [23]:
array[1:].astype(np.float32)

array([  2.67301518e-02,   2.12815736e-04,   5.29392739e-04,
         6.14126388e-04,   9.63569105e-01,   3.16142128e-03,
         1.87458086e-03,   3.30840913e-03], dtype=float32)

In [9]:
min_sample = 2
epsilon = 0.2

In [10]:
def compare(img, img2):
    return np.mean(np.abs(img - img2))

In [13]:
cluster_folder = "/".join(test_path.split("/")[:-2]) + "/test_cluster_{}_{}/".format(min_sample, epsilon)

In [14]:
cluster_folder

'./Submission/test_cluster_2_0.2/'

In [17]:
copytree(test_path, cluster_folder)

'./Submission/test_cluster_2_0.2/'

In [20]:
paths = glob.glob(cluster_folder + "*.jpg")

In [23]:
for path in paths:
    
    folder = path[:-13]
    temp_img = imread(path)
    destination = "{}{}_{}/".format(folder, temp_img.shape[0], temp_img.shape[1])
    mk(destination)
    
    os.rename(path, destination + path[-13:])

In [28]:
# We loop over the folder containing the images.
folders_img_sizes = glob.glob(cluster_folder + "*/")


print("Nb of folders to do:", len(folders_img_sizes))

for folder in glob.glob(cluster_folder + "*/"):
    train_files = glob.glob(folder + "*.jpg")
    train = np.array([imread(img) for img in train_files])


    # Resize the images to speed it up.
    train = [cv2.resize(img, (224, 224), cv2.INTER_LINEAR) for img in train]

    custom_train = []
    for img in train:
        im = (img - img.mean()) / img.std()
        custom_train.append(im.reshape((224*224*3,)))

    train = np.array(custom_train)

    l = len(train)
    
    print(l, "images in", folder,  "to create clusters.")
    
    if l <=4:
        continue
    
    distances = np.zeros((l,l))
    
    # We compute the distance matrix
    for i in range(l):
        for j in range(l):
            distances[i,j] = compare(train[i], train[j])
    
    # We compute the clusters
    cls = cluster.DBSCAN(metric='precomputed', min_samples=3, eps=0.2)
    y = cls.fit_predict(distances)
    
    # We move the images in the good folder.
    for path, cluster_idx in zip(train_files, y.tolist()):
        dest_folder = folder + str(cluster_idx) + "/"
        mk(dest_folder)
        dest_path = dest_folder + path[-13:]
        os.rename(path, dest_path)

Nb of folders to do: 10
45 images in ./Submission/test_cluster_2_0.2\670_1192\ to create clusters.
4 images in ./Submission/test_cluster_2_0.2\700_1244\ to create clusters.
57 images in ./Submission/test_cluster_2_0.2\718_1276\ to create clusters.
606 images in ./Submission/test_cluster_2_0.2\720_1280\ to create clusters.
148 images in ./Submission/test_cluster_2_0.2\750_1280\ to create clusters.
11 images in ./Submission/test_cluster_2_0.2\750_1334\ to create clusters.
21 images in ./Submission/test_cluster_2_0.2\854_1518\ to create clusters.
7 images in ./Submission/test_cluster_2_0.2\924_1280\ to create clusters.
98 images in ./Submission/test_cluster_2_0.2\974_1280\ to create clusters.
3 images in ./Submission/test_cluster_2_0.2\974_1732\ to create clusters.


In [44]:
list_clusters = h_gen.get_all_images(regex=cluster_folder + "*/*/")

In [47]:
for folder in list_clusters:
    
    # We don't want images that don't belong to any cluster
    if folder.split("/")[-2] != "-1":
        
        cluster_points = os.listdir(folder)
        l = len(cluster_points)
        
        # All images belonging to cluster_points should have the same value
        indexes = [dic_submission[filename] for filename in cluster_points]

        average = np.mean(numpy_table[indexes,1:], axis=0)
        
        for i in indexes:
            numpy_table[i,1:] = average

In [51]:
i=150
numpy_table[i:i+10]

array([['img_01142.jpg', 0.2888109702431138, 0.028017524462434357,
        0.0354087180025434, 0.004168763512235246, 0.18127656419199434,
        0.060469898565583075, 0.019658698251600982, 0.3821888627704949],
       ['img_01168.jpg', 0.007980979352309509, 0.025494562687751552,
        0.007761164157996805, 0.005367880887731948, 0.04890710718273972,
        0.8832198128737914, 0.0027186080349547483, 0.0185498848227244],
       ['img_01171.jpg', 0.05975911406780579, 0.012404567070037745,
        9.685317675046703e-05, 0.0005855563676041676, 0.9061719432158168,
        0.013239905493825156, 0.0004108632444721428, 0.007331197363687786],
       ['img_01177.jpg', 0.03500724881624585, 0.11121813500840616,
        0.003313604300807561, 0.0009265952900643225, 0.4952403068349691,
        0.026952738048941962, 0.0014630030542477793, 0.32587836864631725],
       ['img_01179.jpg', 0.7047046788676893, 0.012380567873829138,
        0.06277278994106351, 0.0030327398617407317, 0.1109351291553309,
   

In [36]:
list_clusters[0].split("/")[-2]

'-1'

In [38]:
a = np.array([[5,6,8],[4,3,2],[8,9,6],[3,4,1]])

In [42]:
np.mean(a[[1,3],:], axis=0)

array([ 3.5,  3.5,  1.5])